In [1]:
cd ~/src/Classes/Ansible/

/Users/rick446/src/Classes/Ansible


# Playbook Delegation

Sometimes we may want to run one (or more) tasks on a host that is *not* the targeted host.

For instance, a common need is to do the following three steps when deploying an app server:

- remove a host from the load balancer
- update the code on the host
- add the host to the load balancer

While you *can* run the 'add/remove from load balancer' code on the app server, it's often better to run those API calls from another "control" server.

In [2]:
cat playbooks/delegate-demo.yaml

- hosts: web
  become: true
  tasks:
    - name: Remove host from load balancer
      debug:
        msg: "This would probably be a custom module or the ec2_elb module"
      delegate_to: arborian-06.class.arborian.com
    - name: Deploy some code
      debug:
        msg: "Hey, I'm deploying code!"
    - name: Re-add host to load balancer
      debug:
        msg: "This would probably be a custom module or the ec2_elb module"
      delegate_to: arborian-06.class.arborian.com


In [3]:
%%bash
ansible-playbook -i inventory playbooks/delegate-demo.yaml


PLAY [r-web] *******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-02.class.arborian.com]
ok: [arborian-01.class.arborian.com]

TASK [Remove host from load balancer] ******************************************
ok: [arborian-01.class.arborian.com -> arborian-06.class.arborian.com] => {
    "msg": "This would probably be a custom module or the ec2_elb module"
}
ok: [arborian-02.class.arborian.com -> arborian-06.class.arborian.com] => {
    "msg": "This would probably be a custom module or the ec2_elb module"
}

TASK [Deploy some code] ********************************************************
ok: [arborian-01.class.arborian.com] => {
    "msg": "Hey, I'm deploying code!"
}
ok: [arborian-02.class.arborian.com] => {
    "msg": "Hey, I'm deploying code!"
}

TASK [Re-add host to load balancer] ********************************************
ok: [arborian-01.class.arborian.com -> arborian-

Frequently, delegation is done to the local machine (the machine running Ansible).

In [4]:
cat playbooks/delegate-demo2.yaml

- hosts: web
  become: true
  tasks:
    - name: Remove host from load balancer
      debug:
        msg: "This would probably be a custom module or the ec2_elb module"
      delegate_to: 127.0.0.1
    - name: Deploy some code
      debug:
        msg: "Hey, I'm deploying code!"
    - name: Re-add host to load balancer
      debug:
        msg: "This would probably be a custom module or the ec2_elb module"
      delegate_to: 127.0.0.1


In [6]:
%%bash
ansible-playbook playbooks/delegate-demo2.yaml


PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-02.class.arborian.com]
ok: [arborian-01.class.arborian.com]

TASK [Remove host from load balancer] ******************************************
ok: [arborian-01.class.arborian.com -> 127.0.0.1] => {
    "msg": "This would probably be a custom module or the ec2_elb module"
}
ok: [arborian-02.class.arborian.com -> 127.0.0.1] => {
    "msg": "This would probably be a custom module or the ec2_elb module"
}

TASK [Deploy some code] ********************************************************
ok: [arborian-01.class.arborian.com] => {
    "msg": "Hey, I'm deploying code!"
}
ok: [arborian-02.class.arborian.com] => {
    "msg": "Hey, I'm deploying code!"
}

TASK [Re-add host to load balancer] ********************************************
ok: [arborian-01.class.arborian.com -> 127.0.0.1] => {
    "msg": "This would probably be 

In the special case of delegation to localhost, we can use the `local_action` module:

In [7]:
cat playbooks/delegate-demo3.yaml

- hosts: web
  become: true
  tasks:
    - name: Remove host from load balancer
      local_action:
        module: debug
        msg: "This would probably be a custom module or the ec2_elb module"
    - name: Deploy some code
      debug:
        msg: "Hey, I'm deploying code!"
    - name: Re-add host to load balancer
      local_action:
        module: debug
        msg: "This would probably be a custom module or the ec2_elb module"


In [8]:
%%bash
ansible-playbook playbooks/delegate-demo3.yaml


PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-02.class.arborian.com]
ok: [arborian-01.class.arborian.com]

TASK [Remove host from load balancer] ******************************************
ok: [arborian-01.class.arborian.com -> localhost] => {
    "msg": "This would probably be a custom module or the ec2_elb module"
}
ok: [arborian-02.class.arborian.com -> localhost] => {
    "msg": "This would probably be a custom module or the ec2_elb module"
}

TASK [Deploy some code] ********************************************************
ok: [arborian-01.class.arborian.com] => {
    "msg": "Hey, I'm deploying code!"
}
ok: [arborian-02.class.arborian.com] => {
    "msg": "Hey, I'm deploying code!"
}

TASK [Re-add host to load balancer] ********************************************
ok: [arborian-01.class.arborian.com -> localhost] => {
    "msg": "This would probably be 

## Serializing tasks

By default, Ansible will run tasks in parallel across all hosts targeted. 

If you'd prefer some level of serialization, Ansible provides the `serial` key in plays. The value is the number of hosts to execute in parallel.

In [9]:
cat playbooks/serialize.yaml

- hosts: all
  serial: 1
  tasks:
    - name: Do something to each host, one at a time
      debug: "I am on host {{ansible_host}}"


In [10]:
%%bash
ansible-playbook playbooks/serialize.yaml


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-01.class.arborian.com]

TASK [Do something to each host, one at a time] ********************************
ok: [arborian-01.class.arborian.com] => {
    "msg": "Hello world!"
}

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-02.class.arborian.com]

TASK [Do something to each host, one at a time] ********************************
ok: [arborian-02.class.arborian.com] => {
    "msg": "Hello world!"
}

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-03.class.arborian.com]

TASK [Do something to each host, one at a time] ********************************
ok: [arbori

We can also give Ansible a list of values for `serial` to enable gradual rolling deployments:

In [11]:
cat playbooks/serialize2.yaml

- hosts: all
  serial:
    - 1
    - 20%
    - 100%
  tasks:
    - name: Do something to each host, one at a time
      debug: "I am on host {{ansible_host}}"


In [12]:
%%bash
ansible-playbook playbooks/serialize2.yaml


PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-01.class.arborian.com]

TASK [Do something to each host, one at a time] ********************************
ok: [arborian-01.class.arborian.com] => {
    "msg": "Hello world!"
}

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-02.class.arborian.com]

TASK [Do something to each host, one at a time] ********************************
ok: [arborian-02.class.arborian.com] => {
    "msg": "Hello world!"
}

PLAY [all] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [arborian-03.class.arborian.com]
ok: [arborian-06.class.arborian.com]
ok: [arborian-04.class.arborian.com]
ok: [arborian-05.cl

# Lab

Using `serial` and `delegate_to` (or `local_action`), write a playbook that simulates a rolling deployment of application servers. 

- Start with playbooks/serialize2.yaml
- Add a local_action to add / remove servers from a load balancer (just use the debug module to print a message)

```ansible-playbook -i digital_ocean.py playbooks/serialize2.yaml```


# Some additional notes

A list of "magic" variables can be found at http://docs.ansible.com/ansible/latest/playbooks_variables.html#magic-variables-and-how-to-access-information-about-other-hosts